# Util File Dependency Test

This notebook tests having a file dependency for a file named `util.py`

In [ ]:
import aqueduct
from aqueduct import op, check, metric

import pandas as pd
import os
import numpy as np

In [ ]:
# You can use `localhost` if you're running this notebook on the same machine as the server.
address = "http://localhost:8080"
api_key = aqueduct.get_apikey()
client = aqueduct.Client(api_key, address)

In [ ]:
warehouse = client.integration(name="aqueduct_demo")
customers_table = warehouse.sql(query="SELECT * FROM customers;")
print(type(customers_table))

In [ ]:
customers_table.get().head()

In [ ]:
@op(file_dependencies=["./util.py"])
def log_featurize(cust: pd.DataFrame) -> pd.DataFrame:
    if not os.path.exists("util.py"):
        raise Exception("util.py does not exist!")

    import util

    features = cust.copy()
    skip_cols = util.get_skip_cols()
    for col in features.columns.difference(skip_cols):
        features["log_" + col] = np.log(features[col] + 1.0)
    return features.drop(columns="cust_id")

In [ ]:
features_table = log_featurize(customers_table)

In [ ]:
features_table.get().head()